<a href="https://colab.research.google.com/github/mcampo26/map/blob/master/Project3eoutline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get -qq install python-cartopy python3-cartopy
import cartopy
!pip install pykrige
# Now let's import packages
import pandas as pd
import matplotlib.pyplot as plt
import pylab #using as a plotting library for spatial data, make contours
import numpy as np #gridded datasets and matrices; array package

# import cartopy, which we will use to map
import cartopy.crs as ccrs                   # import projections
import cartopy.feature as cf                 # import features


# import the UniversalKriging function from pykrige.uk 
from pykrige.uk import UniversalKriging


In [5]:
df = pd.read_excel('https://github.com/mcampo26/map/blob/master/FAKEpumpage.xlsx?raw=true')
print(df)

    Pumpage 2018 (gallons/yr)  Pumpage 2019 (gallons/yr)    lam_x    lam_y
0                     4346400                       3750  3513064  3053266
1                     1932000                      93000  3513839  3050592
2                   372610000                     110000  3389816  3121532
3                    77163000                    3687600  3386729  3135944
4                    33228000                     198000  3506761  3068194
5                   380355961                  301516000  3506761  3066874
6                   346099000                     780000  3371537  3085976
7                   166628000                  106710000  3514499  3050592
8                   102275000                  317814000  3460783  3081317
9                   109078000                  212898000  3377663  3065950
10                  130566000                  200000000  3385049  3125841
11                  137053000                   85035000  3447901  3085064
12                   4954

In [0]:
from pyproj import Proj, transform

inProj = Proj('epsg:3035')
outProj = Proj('epsg:4326')
x1,y1 = -11705274.6374,4826473.6922
x2,y2 = transform(inProj,outProj,x1,y1)
print x2,y2